In [1]:
# import xarray as xr
# import matplotlib.pyplot as plt
import numpy as np
# import pyreadr
import json
import pandas as pd
import copy
#Progress bar
# from tqdm import tqdm

Lists from `R` cannot be read in `Python` when saved as `RData` or `rds` files. Even when the `pyreadr` library is used, it is not possible. This library opens both type of file formats, but not if it is a list. To be able to read a list in `Python`, they need to be saved as `json` files in `R` as follows:

```
library(jsonlite)
write_json(l, "test.json")
```

These files can then be opened in `Python` 

```
import json
with open("file.json") as f:
    l = json.load(f)
```

The items inside lists can be loaded as `numpy` matrices or `pandas` data frames.

In [2]:
# r_data = pyreadr.read_r('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/test_LME_4/gridded_params.rds')
# r_data.keys()

In [2]:
with open('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/test_LME_4/gridded_params.json') as f:
    gridded_params = json.load(f)

In [3]:
gridded_params.keys()

dict_keys(['lat', 'lon', 'depth', 'pp', 'r.plank', 'er', 'sst', 'sft', 'Fmort.u', 'Fmort.v', 'min.fishing.size.u', 'min.fishing.size.v', 'effort', 'pref.ben', 'pref.pel', 'det.coupling', 'sinking.rate', 'q0', 'sd.q', 'A.u', 'A.v', 'alpha.u', 'alpha.v', 'def.high', 'def.low', 'K.u', 'K.v', 'K.d', 'AM.u', 'AM.v', 'handling', 'repro.on', 'c1', 'E', 'Boltzmann', 'mu0', 'xs', 'p.s', 'k.sm', 'dx', 'xmin', 'x1', 'x1.det', 'xmax', 'xmax2', 'x', 'Nx', 'ref', 'ref.det', 'Fref.u', 'Fref.v', 'idx', 'tmax', 'delta_t', 'Neq', 'Ngrid', 'U.init', 'V.init', 'W.init', 'equilibrium'])

In [66]:
#Dates used as column names
col_names = pd.date_range('1840-01-01', '1842-01-01', freq = 'MS')

In [83]:
# May need to do this to transform original json data to correct format
# np.transpose(gridded_params['sinking.rate'])
# pp = pd.DataFrame(gridded_params['pp'])
# pp.columns = col_names

# Defining functions called within sizemodel()

In [4]:
# Build a lookup table for diet preference ------
# Looks at all combinations of predator and prey body size: diet preference 
# (in the predator spectrum only)
def phi_f(q, q0, sd_q):
    phi = np.where(q > 0, np.exp(-(q-q0)*(q-q0)/(2*sd_q*sd_q))/(sd_q*np.sqrt(2.0*np.pi)), 0) 
    return phi

# Functions to build lookup tables for (constant) growth
def gphi_f(q1, q0, sd_q):
    return 10**(-q1)*phi_f(q1, q0, sd_q)

# Functions to build lookup tables for (constant) mortality
def mphi_f(q2, q0, sd_q, alpha_u):
    return 10**(alpha_u*q2)*phi_f(q2, q0, sd_q)

# Function to build lookup table for components of 10^(alpha*x)
def expax_f(x, alpha_u):
    return list(10**(np.array(x)*alpha_u))

def gravitymodel(effort, prop_b, depth, n_iter):
    # redistribute total effort across grid cells according to proportion of
    # biomass in that grid cell using gravity model, Walters & Bonfil, 1999, 
    # Gelchu & Pauly 2007, ideal free distribution - Blanchard et al 2008
    for j in range(n_iter):
        suit = prop_b*(1-(np.array(depth)/max(depth)))
        # rescale:
        rel_suit = suit/sum(suit)
        new_effort = effort+(rel_suit*effort)
        mult = sum(effort)/sum(new_effort)
        # gradient drives individuals to best locations at equilibrium (assumed to 
        # reached after 10000 iterations)
        effort = new_effort*mult
    return effort

In [ ]:
# def gridded_sizemodel(params, ERSEM.det.input = F, U_mat, V_mat, W_mat, temp_effect = True,
#                       eps = 1e-5, output = "aggregated", use_init = False):


In [408]:
#From gridded_params json file
pp = np.array(gridded_params['pp'])
r_plank = np.array(gridded_params['r.plank'])
sst = np.array(gridded_params['sst'])
sft = np.array(gridded_params['sft'])
sinking_rate = np.array(gridded_params['sinking.rate'])

[Ngrid] = gridded_params['Ngrid']
[Neq] = gridded_params['Neq']
[q0] = gridded_params['q0']
[sd_q] = gridded_params['sd.q']
[alpha_u] = gridded_params['alpha.u']
[alpha_v] = gridded_params['alpha.v']
U_init = gridded_params['U.init']
V_init = gridded_params['V.init']
[ref] = gridded_params['ref']
[ref_det] = gridded_params['ref.det']
[W_init] = gridded_params['W.init']
[mu0] = gridded_params['mu0']
[k_sm] = gridded_params['k.sm']
[p_s] = gridded_params['p.s']
[xs] = gridded_params['xs']
[Fref_u] = gridded_params['Fref.u']
[Fref_v] = gridded_params['Fref.v']
[Nx] = gridded_params['Nx']
[Fmort_u] = gridded_params['Fmort.u']
[Fmort_v] = gridded_params['Fmort.v']
[Boltzmann] = gridded_params['Boltzmann']
[E] = gridded_params['E']
[c1] = gridded_params['c1']
[A_u] = gridded_params['A.u']
[A_v] = gridded_params['A.v']
[handling] = gridded_params['handling']
[def_high] = gridded_params['def.high']
[K_u] = gridded_params['K.u']
[def_low] = gridded_params['def.low']
[K_v] = gridded_params['K.v']
[repro_on] = gridded_params['repro.on']
[AM_u] = gridded_params['AM.u']
[AM_v] = gridded_params['AM.v']
[K_d] = gridded_params['K.d']

effort = np.array(gridded_params['effort']).astype('float')
effort[0:5,] = effort[0:5,]+10.
effort[5:10,] = effort[5:10,]+2.
effort[10:,] = effort[10:,]+5.

[dx] = gridded_params['dx']
depth = [d for i in gridded_params['depth'] for d in i]
pref_pel = [float(p) for pref in gridded_params['pref.pel'] for p in pref]
pref_ben = [float(b) for pref in gridded_params['pref.ben'] for b in pref]

x = np.array(gridded_params['x'])
y = np.array(gridded_params['x'])

The `Boltzmann` variable has been truncated to 0.0001, but its real value is $8.62e^{-5}$. This has a large effect on growth and mortality calculations, where the temperature effect (`pel_tempeffect`) goes from 2.009482 (as calculated in R) to 59.25972. The effect is similar with `ben_tempeffect` which changes from 0.2442538 to 9.634305.

In [409]:
ui0 = 10**pp

# Initialising matrices

In [410]:
q1  = np.empty([len(x), len(y)])
for i in range(len(y)):
    q1[:, i] = [y[i]-a for a in x]

#q2 is the reverse matrix holding the log(preysize/predatorsize) for all 
#combinations of sizes
q2 = -1*q1

#matrix for recording the two size spectra 
V = np.zeros((Ngrid, len(x), Neq+1))
U = np.zeros((Ngrid, len(x), Neq+1))

#vector to hold detrtitus biomass density (g.m-2)
W = np.zeros((Ngrid, Neq+1))

#matrix for keeping track of growth and reproduction from ingested food:
GG_u = np.zeros((Ngrid, len(x), Neq+1))
R_v = R_u = GG_v = copy.deepcopy(GG_u)

#matrix for keeping track of predation mortality
PM_u = np.zeros((Ngrid, len(x), Neq+1)) 
PM_v = np.zeros((Ngrid, len(x), Neq+1))

#matrix for keeping track of catches  
Y_u = np.zeros((Ngrid, len(x), Neq+1)) 
Y_v = np.zeros((Ngrid, len(x), Neq+1))

#matrix for keeping track of total mortality (Z)
Z_u = np.zeros((Ngrid, len(x), Neq+1))
Z_v = np.zeros((Ngrid, len(x), Neq+1))

#matrix for keeping track of senescence mortality and other (intrinsic) 
#mortality
OM_u = np.zeros(len(x))
SM_v = SM_u = OM_v = copy.deepcopy(OM_u)

#empty vector to hold fishing mortality rates at each size class at time
Fvec_u = np.zeros((Ngrid, len(x), Neq+1))
Fvec_v = np.zeros((Ngrid, len(x), Neq+1))

# Building lookup tables

In [411]:
#lookup tables for terms in the integrals which remain constant over time
gphi  = gphi_f(q1, q0, sd_q)
mphi  = mphi_f(q2, q0, sd_q, alpha_u)

#lookup table for components of 10^(alpha_u*x)
expax = expax_f(x, alpha_u)

# Numerical integration

In [412]:
#For testing only
use_init = True

In [413]:
# set up with the initial values from param - same for all grid cells
#(phyto+zoo)plankton size spectrum (from 0 to ref)
#consumer size spectrum (from ref to 120)
U[0:Ngrid, 0:120, 0] = U_init[0:120] 

# set initial detritivore spectrum  
V[0:Ngrid, (ref_det-1):120, 0] = V_init[(ref_det-1):120]

# set initial detritus biomass density (g.m^-3) 
W[0:Ngrid, 0] = W_init
    
if use_init:
    # set up with the initial values from previous run
    #(phyto+zoo)plankton size spectrum 
    U[0:Ngrid, 0:(ref-1), 0] = U_init[0:(ref-1)]    
    # set initial consumer size spectrum from previous run
    U[0:Ngrid, (ref-1):len(x), 0] = U_init[(ref-1):len(x)]
    # set initial detritivore spectrum from previous run
    V[0:Ngrid, (ref_det-1):len(x), 0] = V_init[(ref_det-1):len(x)]  
    W[0:Ngrid, 0] = W_init

In [414]:
# intrinsic natural mortality
OM_u = mu0*10**(-0.25*np.array(x))
OM_v = mu0*10**(-0.25*np.array(x))

# senescence mortality rate to limit large fish from building up in the system
# same function as in Law et al 2008, with chosen parameters gives similar M2 values as in Hall et al. 2006
SM_u = k_sm*10**(p_s*(np.array(x)-xs))
SM_v = k_sm*10**(p_s*(np.array(x)-xs))

In the chunk below, both `Fref_u` and `Fref_v` are decimal values. `Nx` decreased by one to match results produced by original R code.

In [415]:
#Fishing mortality (THESE PARAMETERS NEED TO BE ESTIMATED!)
# here Fmort_u and Fmort_u= fixed catchability term for U and V to be 
# estimated along with Fref_v and Fref_u
ind_fvecu = np.arange(int(Fref_u-1), (Nx-1))
for i in ind_fvecu:
    Fvec_u[:, i, 0] = Fmort_u*effort[:, 0]

ind_fvecv = np.arange(int(Fref_v-1), (Nx-1))
for i in ind_fvecv:
    Fvec_v[:, i, 0] = Fmort_v*effort[:, 0]

In [416]:
# output fisheries catches per yr at size
Y_u[:, ind_fvecu, 0] = Fvec_u[:, ind_fvecu, 0]*U[:, ind_fvecu, 0]*10**np.array(x)[ind_fvecu]
# output fisheries catches per yr at size
Y_v[:, ind_fvecv, 0] = Fvec_v[:, ind_fvecv, 0]*V[:, ind_fvecv, 0]*10**np.array(x)[ind_fvecv]

In [417]:
temp_effect = True

`Boltzmann` value updated to $8.62e^{-5}$ to match value in R, otherwise, temperature effect results are one order of magnitude higher.

In [418]:
Boltzmann = 8.62e-5

In [432]:
input_w = (sinking_rate[j, i]* 
           (np.sum(defbypred[ref:Nx]*dx)+ 
            np.sum(pel_Tempeffect*OM_u[0:Nx]*det_coupling_u_multiplier)+ 
            np.sum(pel_Tempeffect*SM_u[0:Nx]*det_coupling_u_multiplier))+
           (np.sum(ben_Tempeffect*OM_v[0:Nx]*det_coupling_v_multiplier)+ 
            (np.sum(ben_Tempeffect*OM_v[0:Nx]*det_coupling_v_multiplier)))

SyntaxError: incomplete input (290967042.py, line 6)

In [431]:
det_coupling_u_multiplier = U[j, 0:Nx, i]*10**(x[0:Nx])*dx
np.sum(det_coupling_u_multiplier)

126.37455248423963

In [403]:
for i in range(Neq):
    x_u = 10**np.array(x)[ind_fvecu]*dx
    x_v = 10**np.array(x)[ind_fvecv]*dx
    
    # time
    for i in range(Neq-1):
        # get proportion of total fishable biomass for each grid cell
        # output rates of fisheries catches per yr at size
        prop_bu = np.sum((U[:, ind_fvecu, i]*x_u), axis = 1)
        prop_bv = np.sum((V[:, ind_fvecv, i]*x_v), axis = 1)
        prop_b = (prop_bu+prop_bv)/sum(prop_bu+prop_bv)
        # redistribute total effort across gridcells according to proportion of
        # biomass in that grid cell
        effort[:, i+1] = gravitymodel(effort[:, i+1], prop_b, depth = depth, n_iter = 1)

        for j in np.arange(Ngrid):
            if temp_effect:
                pel_tempeffect = np.exp(c1 - E / (Boltzmann * (sst[j, i] + 273)))
                ben_tempeffect = np.exp(c1 - E / (Boltzmann * (sft[j, i] + 273)))
            else:
                pel_tempeffect = 1
                ben_tempeffect = 1

            f_pel_term = (A_u*10**(x*alpha_u)*pref_pel[j])*((U[j, :, i]*dx)@gphi)
            f_pel = pel_tempeffect*(f_pel_term/(1+handling*f_pel_term))

            f_ben_term = (A_u*10**(x*alpha_u)*pref_ben[j])*((V[j, :, i]*dx)@gphi)
            f_ben = pel_tempeffect*(f_ben_term/(1+handling*f_ben_term))

            f_det_term = ((1/10**x)*A_v*10**(x*alpha_v)*W[j, i])
            f_det = ben_tempeffect*f_det_term/(1+handling*f_det_term)
            del f_ben_term, f_det_term

            GG_u[j, :, i] = (1-def_high)*K_u*f_pel+(1-def_low)*K_v*f_ben

            if repro_on == 1:
                R_u[j, :, i] = (1-def_high)*(1-(K_u+AM_u))*f_pel+(1-def_high)*(1-(K_v+AM_v))*f_ben
                R_v[j, :, i] = (1-def_low)*(1-(K_d+AM_v))*f_det

            sat_pel = [fp/fpt if fp > 0 else 0 for fp, fpt in zip(f_pel, f_pel_term)]

            PM_u[j, :, i] = (pref_pel[j]*A_u*np.array(expax))*((U[j, :, i]*sat_pel*dx)@mphi)

            del f_pel_term

            Z_u[j, :, i] = np.sum(PM_u[j, :, i]+pel_tempeffect*OM_u+SM_u+Fvec_u[j, :, i])
            
            GG_v[j, :, i] = (1-def_low)*K_d*f_det

            sat_ben_divisor = (A_u*10**(x*alpha_v)*pref_ben[j])*((V[j, :, i]*dx)@gphi)
            sat_ben = [fb/sbd if fb > 0 else 0 for fb, sbd in zip(f_ben, sat_ben_divisor)]
            del sat_ben_divisor

            PM_v[j, :, i] = np.where(np.array(sat_ben) > 0, 
                                     (pref_ben[j]*A_u*np.array(expax))*((U[j,:,i]*sat_ben*dx)@mphi), 
                                     0)

            Z_v[j, :, i] = PM_v[j, :, i] + ben_tempeffect * OM_v + SM_v + Fvec_v[j, :, i]
            
            eaten_by_pred = 10**x*f_pel*U[j, :, i]+10**x*f_ben*U[j, :, i]
            
            output_w = np.sum(10**x*f_det*V[j, :, i]*dx)
            
            def_by_pred = def_high*f_pel*10**x*U[j, :, i]+def_low*f_ben*10**x*U[j, :, i]
            
            if ERSEM.det.input == False:
                det_coupling_v_multiplier = V[j, 0:Nx, i]*10**(x[0:Nx])*dx
                if det.coupling == 1.0:
                    det_coupling_u_multiplier = U[j, 0:Nx, i]*10**(x[0:Nx])*dx
                    


/jobfs/123583570.gadi-pbs/ipykernel_1709340/195254654.py:11: RuntimeWarning: invalid value encountered in divide
  prop_b = (prop_bu+prop_bv)/sum(prop_bu+prop_bv)


KeyboardInterrupt: 